<a href="https://colab.research.google.com/github/OsvaldoMS1982/TFM-NLP2SQL/blob/Fase-2/Fase2_EntrenamientodeModelos_T5_Small_NL2SQL_FineTuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Montar Drive

In [ ]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta del dataset Spider en Google Drive
SPIDER_PATH = "/content/drive/My Drive/spider"

# Verificar que los archivos están en la ubicación correcta
print("Archivos en Spider:", os.listdir(SPIDER_PATH))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivos en Spider: ['train_gold.sql', 'dev_gold.sql', 'dev.json', 'train_others.json', 'train_spider.json', 'tables.json', 'README.txt', 'test_tables.json', 'test.json', 'test_gold.sql', '.DS_Store', 'test_database', 'database', 'train_spider_fixed.json']


Cargar los Datos de Spider

In [ ]:
import json

# Cargar datos de entrenamiento y validación
with open(f"{SPIDER_PATH}/train_spider.json", "r") as f:
    train_data = json.load(f)

with open(f"{SPIDER_PATH}/dev.json", "r") as f:
    val_data = json.load(f)

print("Ejemplo de entrenamiento:", train_data[0])
print("Ejemplo de validación:", val_data[0])


Ejemplo de entrenamiento: {'db_id': 'department_management', 'query': 'SELECT count(*) FROM head WHERE age  >  56', 'query_toks': ['SELECT', 'count', '(', '*', ')', 'FROM', 'head', 'WHERE', 'age', '>', '56'], 'query_toks_no_value': ['select', 'count', '(', '*', ')', 'from', 'head', 'where', 'age', '>', 'value'], 'question': 'How many heads of the departments are older than 56 ?', 'question_toks': ['How', 'many', 'heads', 'of', 'the', 'departments', 'are', 'older', 'than', '56', '?'], 'sql': {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}}
Ejemplo de validación: {'db_id': 'concert_singer', 'query': 'SELECT count(*) FROM singer', 'query_toks': ['SELECT', 'count', '(', '*', ')', 'FROM', 'singer'], 'query_toks_no_value': ['select', 'count', '(', '*', ')', 'fr

Preprocesar los Datos para los Modelos

In [ ]:
import pandas as pd

# Función para convertir Spider a formato de entrenamiento
def preprocess_spider(data):
    inputs = []
    targets = []

    for item in data:
        question = item["question"]  # Pregunta en lenguaje natural
        sql_query = item["query"]  # SQL correspondiente

        # Formato de entrada para los modelos
        inputs.append(f"Translate to SQL: {question}")
        targets.append(sql_query)

    return pd.DataFrame({"input": inputs, "target": targets})

# Convertir datos de entrenamiento y validación
train_df = preprocess_spider(train_data)
val_df = preprocess_spider(val_data)

print("Ejemplo de entrada para T5:", train_df.head())


Ejemplo de entrada para T5:                                                input  \
0  Translate to SQL: How many heads of the depart...   
1  Translate to SQL: List the name, born state an...   
2  Translate to SQL: List the creation year, name...   
3  Translate to SQL: What are the maximum and min...   
4  Translate to SQL: What is the average number o...   

                                              target  
0         SELECT count(*) FROM head WHERE age  >  56  
1  SELECT name ,  born_state ,  age FROM head ORD...  
2  SELECT creation ,  name ,  budget_in_billions ...  
3  SELECT max(budget_in_billions) ,  min(budget_i...  
4  SELECT avg(num_employees) FROM department WHER...  


In [ ]:
pip install --upgrade huggingface_hub transformers

Función para Tokenización

In [ ]:

from transformers import AutoTokenizer

def tokenize_data(df, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = tokenizer(df["input"].tolist(), padding="max_length", truncation=True, return_tensors="pt", max_length=512)
    targets = tokenizer(df["target"].tolist(), padding="max_length", truncation=True, return_tensors="pt", max_length=128)
    return inputs, targets, tokenizer


Crear Dataset PyTorch

In [ ]:
import torch
from torch.utils.data import Dataset

class SpiderDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.targets["input_ids"][idx]
        }


Configurar y Entrenar los Modelos T5-LM-Large-text2sql-spider

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback


# Modelos en Hugging Face
models = {
    "T5-Small-NL2SQL": "Shritama/t5-small-finetuned-nl2sql",
    #"BART-LARGE-NL2SQL": "SwastikM/bart-large-nl2sql",
    #"NL2SQL-StarCoder-15B": "gabrielpondc/NL2SQL-StarCoder-15B",
  #  "AutoSQL-nl2sql-8b": "xbrain/AutoSQL-nl2sql-1.0-8b",
   # "T5-Small-NL2SQL": "Shritama/t5-small-finetuned-nl2sql"
}

# Entrenar cada modelo
for model_name, model_path in models.items():
    print(f"\n🔵 Entrenando {model_name}...\n")

    # Tokenizar datos
    train_inputs, train_targets, tokenizer = tokenize_data(train_df, model_path)
    val_inputs, val_targets, _ = tokenize_data(val_df, model_path)

    # Crear dataset
    train_dataset = SpiderDataset(train_inputs, train_targets)
    val_dataset = SpiderDataset(val_inputs, val_targets)

    # Cargar modelo
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

    # Configurar entrenamiento
    training_args = TrainingArguments(
        #Fine Tune: se agrega fp16=True para reducir memoria y mejorar el entrenamiento
        fp16=True,
        output_dir=f"./{model_name}_spider",
        evaluation_strategy="epoch",
        #Fine Tune: Incrementamos el trainin batch y eval batch de 4 a 8 para mejorar el entrenamiento
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        #Fine Tune reducimos el learning_rate=3e-5,para hacerlo mas estable
        learning_rate=1e-5,
        weight_decay=0.01,
        load_best_model_at_end=True, #Requerido para Early Stopping
        #Aumentamos cantidad de epocas de 3 a 5, ahora incrementamos a 7
        num_train_epochs=7,
        logging_dir=f"./logs/{model_name}",
        save_strategy="epoch"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        #optimizers=(trainer.optimizer, lr_scheduler)
    )

    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))


    # Iniciar entrenamiento
    trainer.train()

    # Guardar modelo en Google Drive
    save_path = f"/content/drive/My Drive/spider_models_fine_Tuned/{model_name}"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"✅ Modelo {model_name} guardado en {save_path}")



🔵 Entrenando T5-Small-NL2SQL...



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.464700,0.699342
2,0.729700,0.609927
3,0.601500,0.576252
4,0.550700,0.554152
5,0.525200,0.551633
6,0.510700,0.544431
7,0.502800,0.541947


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


✅ Modelo T5-Small-NL2SQL guardado en /content/drive/My Drive/spider_models_fine_Tuned/T5-Small-NL2SQL


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback


# Modelos en Hugging Face
models = {
    "T5-Small-NL2SQL": "Shritama/t5-small-finetuned-nl2sql",
    #"BART-LARGE-NL2SQL": "SwastikM/bart-large-nl2sql",
    #"NL2SQL-StarCoder-15B": "gabrielpondc/NL2SQL-StarCoder-15B",
  #  "AutoSQL-nl2sql-8b": "xbrain/AutoSQL-nl2sql-1.0-8b",
   # "T5-Small-NL2SQL": "Shritama/t5-small-finetuned-nl2sql"
}

# Entrenar cada modelo
for model_name, model_path in models.items():
    print(f"\n🔵 Entrenando {model_name}...\n")

    # Tokenizar datos
    train_inputs, train_targets, tokenizer = tokenize_data(train_df, model_path)
    val_inputs, val_targets, _ = tokenize_data(val_df, model_path)

    # Crear dataset
    train_dataset = SpiderDataset(train_inputs, train_targets)
    val_dataset = SpiderDataset(val_inputs, val_targets)

    # Cargar modelo
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

    # Configurar entrenamiento
    training_args = TrainingArguments(
        #Fine Tune: se agrega fp16=True para reducir memoria y mejorar el entrenamiento
        fp16=True,
        output_dir=f"./{model_name}_spider",
        evaluation_strategy="epoch",
        #Fine Tune: Incrementamos el trainin batch y eval batch de 4 a 8 para mejorar el entrenamiento
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        #Fine Tune reducimos el learning_rate=3e-5,para hacerlo mas estable
        learning_rate=1e-5,
        weight_decay=0.01,
        load_best_model_at_end=True, #Requerido para Early Stopping
        #Aumentamos cantidad de epocas a 9
        num_train_epochs=9,
        logging_dir=f"./logs/{model_name}",
        save_strategy="epoch"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        #optimizers=(trainer.optimizer, lr_scheduler)
    )

    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))


    # Iniciar entrenamiento
    trainer.train()

    # Guardar modelo en Google Drive
    save_path = f"/content/drive/My Drive/spider_models_fine_Tuned/{model_name}"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"✅ Modelo {model_name} guardado en {save_path}")



🔵 Entrenando T5-Small-NL2SQL...



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.461600,0.697234
2,0.722600,0.610363
3,0.588800,0.570234
4,0.534300,0.543981
5,0.506500,0.543631
6,0.486200,0.536489
7,0.470900,0.526316
8,0.455200,0.520637
9,0.451000,0.519721


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


✅ Modelo T5-Small-NL2SQL guardado en /content/drive/My Drive/spider_models_fine_Tuned/T5-Small-NL2SQL


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback


# Modelos en Hugging Face
models = {
    "T5-Small-NL2SQL": "Shritama/t5-small-finetuned-nl2sql",
    #"BART-LARGE-NL2SQL": "SwastikM/bart-large-nl2sql",
    #"NL2SQL-StarCoder-15B": "gabrielpondc/NL2SQL-StarCoder-15B",
  #  "AutoSQL-nl2sql-8b": "xbrain/AutoSQL-nl2sql-1.0-8b",
   # "T5-Small-NL2SQL": "Shritama/t5-small-finetuned-nl2sql"
}

# Entrenar cada modelo
for model_name, model_path in models.items():
    print(f"\n🔵 Entrenando {model_name}...\n")

    # Tokenizar datos
    train_inputs, train_targets, tokenizer = tokenize_data(train_df, model_path)
    val_inputs, val_targets, _ = tokenize_data(val_df, model_path)

    # Crear dataset
    train_dataset = SpiderDataset(train_inputs, train_targets)
    val_dataset = SpiderDataset(val_inputs, val_targets)

    # Cargar modelo
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

    # Configurar entrenamiento
    training_args = TrainingArguments(
        #Fine Tune: se agrega fp16=True para reducir memoria y mejorar el entrenamiento
        fp16=True,
        output_dir=f"./{model_name}_spider",
        evaluation_strategy="epoch",
        #Fine Tune: Incrementamos el trainin batch y eval batch de 4 a 8 para mejorar el entrenamiento
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        #Fine Tune reducimos el learning_rate=3e-5,para hacerlo mas estable
        learning_rate=1e-5,
        weight_decay=0.01,
        load_best_model_at_end=True, #Requerido para Early Stopping
        #Aumentamos cantidad de epocas a 15
        num_train_epochs=15,
        logging_dir=f"./logs/{model_name}",
        save_strategy="epoch"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        #optimizers=(trainer.optimizer, lr_scheduler)
    )

    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))


    # Iniciar entrenamiento
    trainer.train()

    # Guardar modelo en Google Drive
    save_path = f"/content/drive/My Drive/spider_models_fine_Tuned/{model_name}"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"✅ Modelo {model_name} guardado en {save_path}")



🔵 Entrenando T5-Small-NL2SQL...



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.458800,0.693660
2,0.714300,0.604788
3,0.574300,0.563985
4,0.514800,0.540948
5,0.484700,0.534806
6,0.459000,0.521794
7,0.437100,0.514586
8,0.414500,0.504611
9,0.406200,0.496596
10,0.399700,0.488796


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


✅ Modelo T5-Small-NL2SQL guardado en /content/drive/My Drive/spider_models_fine_Tuned/T5-Small-NL2SQL


Probar el Modelo Entrenado

In [ ]:
!pip install sacrebleu datasets torch tabulate

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# 📌 Lista de modelos a evaluar
models_to_test = {

    "T5-Small-NL2SQL": "/content/drive/MyDrive/spider_models_fine_Tuned/T5-Small-NL2SQL",

}

# 📌 Diccionario para almacenar modelos y tokenizers
models_dict = {}

for model_name, model_path in models_to_test.items():
    try:
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        models_dict[model_name] = {"model": model, "tokenizer": tokenizer}
        print(f"✅ {model_name} cargado correctamente.")
    except Exception as e:
        print(f"❌ Error al cargar {model_name}: {e}")


✅ T5-Small-NL2SQL cargado correctamente.


Probar el Modelo

In [ ]:
# Seleccionar un modelo entrenado
model_name = "T5-Small-NL2SQL"
model_path = f"/content/drive/My Drive/spider_models_fine_Tuned/{model_name}"

# Cargar modelo y tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Función de inferencia
def generate_sql(question):
    input_text = f"Translate to SQL: {question}"
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generar SQL
    output = model.generate(**inputs)
    sql_query = tokenizer.decode(output[0], skip_special_tokens=True)
    return sql_query

# Prueba con una pregunta nueva
#question = "¿Cuántos empleados hay en la base de datos?"
question = "count of employees?"
sql_generated = generate_sql(question)

print("Pregunta:", question)
print("SQL Generado:", sql_generated)


Pregunta: count of employees?
SQL Generado: SELECT count(*) FROM employees


Generar Predicciones SQL para Validación

In [ ]:
import json

SPIDER_PATH = "/content/drive/My Drive/spider"

# 📌 Cargar datos de validación de Spider
with open(f"{SPIDER_PATH}/dev.json", "r") as f:
    val_data = json.load(f)

# 📌 Extraer preguntas y sus SQL correctos
questions = [item["question"] for item in val_data]
true_sql = [item["query"] for item in val_data]

# 📌 Función para generar SQL con cada modelo
def generate_sql(model, tokenizer, question):
    input_text = f"Translate to SQL: {question}"
    inputs = tokenizer(input_text, return_tensors="pt")

    output = model.generate(
        **inputs, max_length=128, temperature=0.7, top_p=0.9, num_return_sequences=1, repetition_penalty=1.2
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 📌 Generar consultas SQL con cada modelo
predictions = {}
for model_name, data in models_dict.items():
    print(f"\n🚀 Generando SQL con {model_name}...\n")
    model = data["model"]
    tokenizer = data["tokenizer"]
    predictions[model_name] = [generate_sql(model, tokenizer, q) for q in questions]



🚀 Generando SQL con T5-Small-NL2SQL...



/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Evaluar Modelo

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00


In [ ]:
import evaluate
import sqlite3
from tabulate import tabulate

# 📌 Cargar métricas BLEU
bleu_metric = evaluate.load("sacrebleu")

# 📌 Función para calcular Exact Match
def exact_match(pred, true):
    return int(pred.strip().lower() == true.strip().lower())

# 📌 Función para ejecutar consultas SQL en SQLite
def execute_sql(query, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()
    except Exception as e:
        return None  # Si la consulta es inválida, devuelve None

# 📌 Función para calcular Execution Accuracy
def execution_accuracy(predictions, references, conn):
    correct = sum(1 for pred, ref in zip(predictions, references) if execute_sql(pred, conn) == execute_sql(ref, conn))
    return (correct / len(references)) * 100

# 📌 Crear una base de datos en memoria para evaluar Execution Accuracy
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 🚀 Simulación de una base de datos (Asegúrate de usar una DB real en producción)
cursor.execute("CREATE TABLE employees (id INTEGER, name TEXT, salary INTEGER)")
cursor.executemany("INSERT INTO employees (id, name, salary) VALUES (?, ?, ?)", [
    (1, "Alice", 50000),
    (2, "Bob", 60000),
    (3, "Charlie", 70000)
])
conn.commit()

# 📌 Evaluar cada modelo
results = []

for model_name in models_dict.keys():
    pred_sql = predictions[model_name]

    # 📌 Calcular BLEU Score
    bleu_score = bleu_metric.compute(predictions=pred_sql, references=[[x] for x in true_sql])["score"]

    # 📌 Calcular Exact Match (EM)
    em_score = sum(exact_match(pred_sql[i], true_sql[i]) for i in range(len(true_sql))) / len(true_sql)

    # 📌 Calcular Execution Accuracy
    exec_acc = execution_accuracy(pred_sql, true_sql, conn)

    results.append([model_name, round(bleu_score, 2), round(em_score * 100, 2), round(exec_acc, 2)])

# 📌 Mostrar Resultados en Tabla
print("\n📊 **Comparación de Métricas**\n")
print(tabulate(results, headers=["Modelo", "BLEU Score", "Exact Match (%)", "Execution Accuracy (%)"], tablefmt="grid"))

# 📌 Cerrar la conexión con la base de datos
conn.close()



📊 **Comparación de Métricas**

+-----------------+--------------+-------------------+--------------------------+
| Modelo          |   BLEU Score |   Exact Match (%) |   Execution Accuracy (%) |
+=================+==============+===================+==========================+
| T5-Small-NL2SQL |        21.46 |              1.35 |                    99.81 |
+-----------------+--------------+-------------------+--------------------------+


1️⃣ BLEU Score (21.46) es aceptable, pero bajo comparado con modelos más avanzados (35+).

2️⃣ Exact Match (1.35%) es extremadamente bajo, lo que indica que el modelo rara vez genera consultas idénticas a las referencias.

3️⃣ Execution Accuracy (99.81%) es prácticamente perfecto, lo que significa que las consultas generadas producen los mismos resultados que las referencias en la base de datos.

✅ El modelo está generalizando bien y generando SQL válido.
⚠ No es necesario obsesionarse con Exact Match, porque la consulta puede ser diferente en texto pero funcionalmente correcta.